# Scan Time and Signal to Noise Ratio (SNR)

The scan time and signal to noise ratio (SNR) are critical characteristics that impacts image quality and what is practical in MRI.  For MRI, there is a strong theoretical foundation for the SNR dependency based on the MRI physics and pulse sequence characteristics.  

## Learning Goals

1. Understand the constraints and tradeoffs in MRI
    * Determine how long a scan takes
    * Describe the factors that influence image SNR
1. Manipulate MRI sequence parameters to improve performance
    * Understand how modifications in sequence parameters will affect the total scan time and expected SNR

## Scan Time

The total scan time, $T_{scan}$, depends on the length of each component in the pulse sequence as well as how many times these components are repeated.  

For sequences with a single component of length $TR$, this can be expressed as

$$T_{scan} = TR \ N_{TR}$$

where $N_{TR}$ is the total number of TRs which can depend on the number of phase encodes, preparation or dummy cycles to get into the steady-state, acceleration factor, and number of averages.

## SNR Dependencies

The SNR in factors we control in an MRI experiment can be decomposed into dependencies on three factors:

1. Voxel volume
2. Data acquisition time
3. Transverse magnetization

There are other system factors that influence SNR such as the magnetic field strength and the RF coils.

### Voxel Volume

The MRI signal is an integral over a volume of the transverse magnetization.  Therefore there is a linear dependency on the MRI signal and the volume of magnetization being examined.  So we have

$$SNR \propto \mathrm{Voxel\ Volume} = \delta_x\ \delta_y\ \delta_z$$

where $\delta_x, \delta_y, \delta_z$ are the voxel dimensions in x, y, and z.

### Data Acquisition Time

Every additional measurement allows for signal averaging since the noise varies at each measurement.  Note that MRI noise is zero-mean with a Gaussian distribution.  Additional data acquisition time increases signal linearly, while the total noise increases as the square root.  So

$$SNR \propto \frac{S}{\sigma} = \frac{T_{meas}}{\sqrt{T_{meas}}} = \sqrt{T_{meas}}$$

where $T_{meas}$ is the _total data acquisition/measurement time_ including all TRs and signal averaging, but not including the sequence downtime.  This can be defined in terms of the readout duration per TR, $T_{read}$, and the total number of TRs, $N_{TR}$, which for example includes number of ,phase encoding steps and signal averages, as:

$$T_{meas} = N_{TR} T_{read}$$

### Transverse Magnetization

The MRI signal is also proportional to the transverse magnetization, including modulation that occurs due to pulse sequence parameters such as TE, TR, flip angle, spin-echo versus gradient-echo, etc.  This is then also varying depending on the MR properties such as proton density, relaxation rates, diffusion coefficients, etc.  Since this varies widely by pulse sequence, we summarize this as

$$SNR \propto f_{seq}$$

For example, in a simple spoiled gradient-echo pulse sequence, 

$$f_{seq,GRE} = M_0 \sin(\theta) \frac{1- \exp(-TR/T_1)}{1- \cos(\theta) \exp(-TR/T_1)}$$



## SNR Equations

Putting it all together,

$$SNR \propto f_{seq}\ \mathrm{Voxel\ Volume}\ \sqrt{T_{meas}}$$

**This simple equation describes the majority of SNR effects in MRI, and is extremely powerful when designing and improving MRI scans!**

### Alternative formulations of SNR

MRI scanner interfaces often do not provide direct control of the resolutions or measurement times, but instead parameters such as matrix sizes, field-of-views (FOVs) and readout bandwidths are used.  The following relationships can generally be used to convert these parameters into the SNR equation above.

$$ FOV = \delta \ N_{matrix} $$

$$ T_{read} = \frac{N_{readout}}{BW} = \frac{N_{readout}}{2 BW_{one-sided}} $$

If acceleration methods such as partial Fourier, parallel imaging, or compressed sensing are NOT used, the following relationships also apply

$$ T_{read} = \frac{N_{FE}}{BW} = \frac{1}{BW/pixel} $$

$$ N_{TR} = N_{FE} \ N_{PE} \ N_{averages} $$

However, partial Fourier methods can potentially be used to reduce the readout time or number of encoding steps, and parallel imaging/compressed sensing can be used to reduce the number of encoding steps.

## SNR Efficiency

The SNR efficiency for MRI provides a measure of overall efficiency within a MRI scan, which is the SNR normalized by the *total scan time*, $T_{scan}$

$$SNR_{efficiency} = \frac{SNR}{\sqrt{T_{scan}}} \propto f_{seq}\ \mathrm{Voxel\ Volume}\ \sqrt{\frac{T_{meas} }{T_{scan}}}$$

## Resolution and SNR in Post-Processing

The image resolution is often changed in post-processing, and can be either upsampled to finer ("higher") spatial resolution or downsampled to coarser ("lower") spatial resolution.  When using the SNR equation above, the acquired resolution should be considered.

### Upsampling

Upsampling is typically done with some type of interpolation.  In MRI, this can be easily implemented in k-space through zero-padding, which is equivalent to sinc interpolation.  Upsampling with a reasonable method does not affect SNR.

### Downsampling

Downsampling is done to increase SNR by changing the image to a coarser resolution.  However, downsampling is effectively averaging signals from nearby voxels which also averages out the noise.  Therefore, *downsampling scales SNR as the square root of the voxel size, similarly to the measurement time*. 

This is a very important distinction from changed the acquired resolution, where the SNR scales linearly with voxel size.  Therefore, the SNR is higher when acquiring at the desired spatial resolution compared to acquiring at a finer resolution and then downsampling.  Care should be taken to choose the acquired resolution to match the desired resolution.